In [1]:
import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility

import keras
import tensorflow as tf
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Bidirectional

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Using TensorFlow backend.


In [2]:
print 'loading data'
trainmat = h5py.File('deepsea_data/train.mat')
validmat = scipy.io.loadmat('deepsea_data/valid.mat')
testmat = scipy.io.loadmat('deepsea_data/test.mat')

X_train = np.transpose(np.array(trainmat['trainxdata']),axes=(2,0,1))
y_train = np.array(trainmat['traindata']).T

loading data


In [ ]:
forward_lstm = LSTM(units=320, return_sequences=True)
# backward_lstm = LSTM(input_dim=320, output_dim=320, return_sequences=True)
brnn = Bidirectional(forward_lstm)

print 'building model'

model = Sequential()
model.add(Convolution1D(activation="relu", 
                        input_shape=(1000, 4), 
                        padding="valid", strides=1, 
                        filters=320, kernel_size=26))

model.add(MaxPooling1D(strides=13, pool_size=13))

model.add(Dropout(0.2))

model.add(brnn)

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(input_dim=75*640, units=925))
model.add(Activation('relu'))

model.add(Dense(input_dim=925, units=919))
model.add(Activation('sigmoid'))

print 'compiling model'
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['cosine'])

print 'running at most 60 epochs'

checkpointer = ModelCheckpoint(filepath="DanQ_bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.fit(X_train, y_train, batch_size=2048, epochs=60, 
          shuffle=True,
          validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), 
          callbacks=[checkpointer,earlystopper])

In [5]:
tresults = model.evaluate(np.transpose(testmat['testxdata'],axes=(0,2,1)), 
                          testmat['testdata'])

print tresults

455024/455024 [==============================] - 72s 158us/step
[0.06152647561230429, -0.30007137424549335]


In [7]:
preds = model.predict(np.transpose(testmat['testxdata'],axes=(0,2,1)))
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [13]:
preds_flat = preds.flatten()
test_flat = testmat['testdata'].flatten()

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print(accuracy_score(test_flat, preds_flat))
print(confusion_matrix(test_flat, preds_flat))

0.9812509859695882
[[409043292    529472]
 [  7310748   1283544]]
